In [1]:
import json
import pandas as pd       
import os
import nltk
from nltk.corpus import stopwords

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
##context = 10          # Context window size   - SkipGram                                                                                 
#context = 5          # Context window size     - CBOW                                                                             
#downsampling = 1e-3   # Downsample setting for frequent words

# Load the model that we created in Part 2 
from gensim.models import Word2Vec
import os


# Open Trining File
os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/IN/")
train = pd.read_csv("TRAIN_classifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)
test  = pd.read_csv("TEST_NOclassifiedFULL_NOADMISS.tsv", header=0, delimiter="^", quoting=3)

os.chdir("/temp/DocumentCorpus/ECHR/echr_dataset-master/echr_dataset-master/document_crawler/OUT/AV3/CV/")

train.shape
test.shape

train.columns.values

#model = Word2Vec.load("300features_10minwords_10context")
#model = Word2Vec.load("CBOW_300features_10minwords_10context")
#model = Word2Vec.load("CBOW_300features_10minwords_5context")
model = Word2Vec.load("SG_300features_10minwords_5context")

type(model.wv.syn0)

model.wv.syn0.shape

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5a. Remove extra Stop words
    moreStopwords = ["b","full","value","file","period","months","additional","basis","street","respect","cases","application","amount","since","costs","address","well","days","series","c","particular","purposes","purpose","proceedings","proceeding","h","able","ba","country","held","board","first","second","final","judge","non","statement","documents","many","notes","note","j","considered","aw","echr","whether","language","ill","time","taken","kh","rovd","must","set","within","p","mr","mrs","provided","sher","one","new","route","routes","three","would","previously","shall","en","k","g","applicants","eur","date","might","paragraph","u","kd","could","made","company","see","public","parking","statements","article","government","business","information","therefore","right","also","applicant","court","act","state","security","section","hearing","v","service","case","law","person","courts","regional","january","february","march","april","may","june","july","august","september","october","november","december"]
    more_meaningful_words = [token for token in words if token not in moreStopwords]
    #
    # 5. Return a list of words
    return(more_meaningful_words)
    #return(words)


import nltk.data
#nltk.download()   

import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    # raw_sentences = tokenizer.tokenize(review.strip())
    raw_sentences = tokenizer.tokenize(review.strip().decode("utf8"))
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec
    


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
     
    for review in reviews:
       # Print a status message every # review
       if counter%100. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

import time

start = time.time() # Start time

clean_train_reviews = []
for review in train["sentences"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test sentences"
clean_test_reviews = []
for review in test["sentences"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

end = time.time()
elapsed = end - start
print "Time taken for Creating average feature vecs for test sentences: ", elapsed, "seconds."

C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 4000
Review 100 of 4000
Review 200 of 4000
Review 300 of 4000
Review 400 of 4000
Review 500 of 4000
Review 600 of 4000
Review 700 of 4000
Review 800 of 4000
Review 900 of 4000
Review 1000 of 4000
Review 1100 of 4000
Review 1200 of 4000
Review 1300 of 4000
Review 1400 of 4000
Review 1500 of 4000
Review 1600 of 4000
Review 1700 of 4000
Review 1800 of 4000
Review 1900 of 4000
Review 2000 of 4000
Review 2100 of 4000
Review 2200 of 4000
Review 2300 of 4000
Review 2400 of 4000
Review 2500 of 4000
Review 2600 of 4000
Review 2700 of 4000
Review 2800 of 4000
Review 2900 of 4000
Review 3000 of 4000
Review 3100 of 4000
Review 3200 of 4000
Review 3300 of 4000
Review 3400 of 4000
Review 3500 of 4000
Review 3600 of 4000
Review 3700 of 4000
Review 3800 of 4000
Review 3900 of 4000
Creating average feature vecs for test sentences
Review 0 of 2348
Review 100 of 2348
Review 200 of 2348
Review 300 of 2348
Review 400 of 2348
Review 500 of 2348
Review 600 of 2348
Review 700 of 2348
Review 800 of

In [2]:
#from tempfile import TemporaryFile
#outfile = trainDataVecs
#
## Fit a random forest to the training data, using 100 trees
#from sklearn.ensemble import RandomForestClassifier
#forest = RandomForestClassifier( n_estimators = 100 )
#
#print "Fitting a random forest to labeled training data..."
#forest = forest.fit( trainDataVecs, train["AV3"] )
#
## Test & extract results 
#result = forest.predict( testDataVecs )

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

grid_values = {} #{'C':[30]} # Decide which settings you want for the grid search. 

model_LR = GridSearchCV(RandomForestClassifier(n_estimators = 100), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 

# Try to set the scoring on what the contest is asking for. 
# The contest says scoring is for area under the ROC curve, so use this.
                            
model_LR.fit(trainDataVecs, train["AV3"] ) # Fit the model.

model_LR.grid_scores_

## Write the test results 
#output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV3predict":result} )
##output.to_csv( "Word2Vec_AverageVectors_300features_10minwords_10context.csv", index=False, quoting=3, escapechar='\\', sep='^' )
#output.to_csv( "Word2Vec_CBOW_AverageVectors_300features_10minwords_10context.csv", index=False, quoting=3, escapechar='\\', sep='^' )

LR_result = model_LR.predict_proba(testDataVecs)[:,1] 
#LR_output = pd.DataFrame(data={"id":test["id"], "AV3":LR_result}) # Create our dataframe that will be written.
LR_output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV3_Predict":LR_result} )
#LR_output.to_csv('Logistic_Reg_Proj2.csv', index=False, quoting=3) # Get the .csv file we will submit to Kaggle.
LR_output.to_csv( "A3008_SG_AVG_NSX.csv", index=False, quoting=3, escapechar='\\', sep='^'  )






C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\danny\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

import time

idx=10000000000000
start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th (for 5 times the vocab size, use 1/25th ) of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 25

idx=0
batch_size=45
# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters, random_state=39, init_size=10758)
#kmeans_clustering = MiniBatchKMeans(n_clusters=num_clusters)
#kmeans_clustering = MiniBatchKMeans(init='k-means++', n_clusters=num_clusters, batch_size=batch_size, n_init=10, max_no_improvement=10, verbose=0)
##idx = kmeans_clustering.fit_transform(word_vectors)
#idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."


joblib.dump(kmeans_clustering,  'doc_cluster.pkl')

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))


Time taken for K Means clustering:  145.157999992 seconds.


In [4]:
model.most_similar("arrest")

[(u'lawful', 0.8448737859725952),
 (u'effecting', 0.8442620038986206),
 (u'fleeing', 0.816905677318573),
 (u'vagrants', 0.7921059727668762),
 (u'arrested', 0.7879447937011719),
 (u'insurrection', 0.7840983867645264),
 (u'arresting', 0.7816579341888428),
 (u'quelling', 0.7780519127845764),
 (u'committing', 0.7724398970603943),
 (u'arrestee', 0.7697550654411316)]

In [5]:
model.similarity('russia','ireland' )

0.48704844215384302

In [6]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( np.any(word_centroid_map.values()[i] == cluster) ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'sensitivity', u'massive', u'sides', u'circular', u'skin', u'blurred', u'deep', u'enlargement', u'closed']

Cluster 1
[u'yoshkar', u'ola', u'vake', u'bitola', u'mtatsminda', u'thessaloniki', u'cani', u'taganrog', u'saburtalo', u'tbilisi', u'pils', u'komi', u'balkariya', u'dar', u'skopje', u'altay', u'chernigiv', u'krtsanisi', u'presnenskiy', u'oto', u'klaip']

Cluster 2
[u'unfit', u'diver', u'sickness', u'considerably', u'fewer', u'degrees', u'exacerbated', u'increased', u'compared', u'extra', u'comparison', u'low', u'lowered', u'significantly', u'decrease', u'usual', u'decreasing', u'level', u'mortality', u'decompression', u'gradually', u'fold', u'doubled', u'saturation']

Cluster 3
[u'journals', u'engagement', u'shadow', u'remembrance', u'ets', u'analytical', u'sects', u'parliamentarians', u'controversial', u'publicised', u'democracies', u'devoted', u'reacting', u'describes', u'terminology', u'conscientious', u'agendas', u'broadcasters', u'affirming', u'kong', u'topics',

In [7]:
############# SkipGram
#Cluster 0
#[u'exposure', u'nuisance', u'disruption', u'climate', u'degradation', u'disturbance']
#
#Cluster 1
#[u'necessities', u'quotas', u'logged', u'milk', u'utensils', u'eggs', u'voltage']
#
#Cluster 2
#[u'rauscher']
#
#Cluster 3
#[u'boat', u'motorway', u'morgue', u'deserted', u'lake']
#
#Cluster 4
#[u'zdrowia', u'zak', u'wolno', u'zdrowotnej', u'karnych', u'aresztach', u'trybu', u'dla']
#
#Cluster 5
#[u'cellmate', u'colleague', u'neighbour', u'friend', u'stabbing', u'fellow', u'girlfriend', u'disbarment']
#
#Cluster 6
#[u'former', u'war', u'socialist', u'peacekeeping', u'repressions', u'soviet', u'proclaimed', u'krajina']
#
#Cluster 7
#[u'chewing', u'slightest', u'detachment', u'bowel', u'persisting', u'volatile', u'airways', u'respiration', u'ocd', u'worst']
#
#Cluster 8
#[u'dniprovskyy', u'sholokhovskiy', u'zamo', u'poprad']
#
#Cluster 9
#[u'launching', u'initiation', u'instituting', u'commencing', u'conducting', u'initiating']



##################### CBOW
#Cluster 0
#[u'defeat', u'akkum', u'exacerbate', u'displace', u'undeniably']
#
#Cluster 1
#[u'nicknamed']
#
#Cluster 2
#[u'watts']
#
#Cluster 3
#[u'lbro', u'alena', u'vilanova', u'pejchal', u'pastor', u'wojtyczek', u'silvis', u'koskelo', u'mahoney', u'dmitry', u'lubarda', u'faris', u'krzysztof', u'robert', u'pauliine', u'keller', u'vehabovi', u'helen', u'gabriele', u'serghides', u'pere', u'spano', u'fridrik', u'antoanella', u'armen', u'kj', u'gaetano', u'georgios', u'potocki', u'kucsko', u'lemmens', u'stadlmayer', u'albuquerque', u'pol', u'branko', u'dedov', u'iulia', u'egidijus', u'ale', u'harutyunyan', u'johannes', u'ris', u'gri', u'motoc', u'jon', u'njak', u'valeriu']
#
#Cluster 4
#[u'dzhamayev', u'dzhabayev', u'malikov', u'khadayev', u'sun', u'sagayev', u'khanpasha', u'salu', u'mansur', u'temersultanov', u'sadykov', u'elmarzayev', u'ilias', u'yunadi', u'guluyev']
#
#Cluster 5
#[u'tree', u'iron', u'device', u'unheated', u'empty', u'electrical', u'immersion', u'expensive', u'automatic', u'implant', u'electric']
#
#Cluster 6
#[u'believes', u'emphasising', u'shows', u'inconceivable', u'indicates', u'suggests', u'supports', u'maintains']
#
#Cluster 7
#[u'sensory', u'faculties', u'pressures', u'ease', u'psychosocial', u'progressive', u'deficiency', u'physiological', u'canonical', u'intellectual', u'resource', u'cognitive', u'reserves', u'impairments', u'functional', u'sufferings', u'emergencies', u'growth']
#
#Cluster 8
#[u'priozersk', u'bardejov']
#
#Cluster 9
#[u'submit', u'provide', u'produce', u'furnish', u'enclose', u'disclose']


#### CBOW with NO Stop words Model
#Cluster 0
#[u'waziristan', u'shelters', u'originates', u'labor', u'mosul', u'castle', u'bosasso', u'lampedusa', u'batticaloa', u'pledged', u'kurd', u'arabs', u'eradicated', u'hia', u'migrant', u'khost', u'asian', u'inhabited', u'damascus', u'settlements', u'cspa', u'satsita', u'graves', u'ajanib', u'diya', u'eelam', u'irregularly', u'georgians', u'socialised', u'native', u'strongholds', u'census', u'undocumented', u'tens', u'abkhazia', u'governorate', u'repressed']
#
#Cluster 1
#[u'xix', u'ohchr', u'dd', u'unami', u'wherein', u'caj', u'ogn']
#
#Cluster 2
#[u'manner', u'way']
#
#Cluster 3
#[u'destroy', u'destroying', u'tampering', u'procuring', u'tamper']
#
#Cluster 4
#[u'disguised', u'dividends']
#
#Cluster 5
#[u'sustaining', u'hymen', u'trauma', u'massive', u'signs', u'bacterium', u'craniocerebral', u'thrombosis', u'traumas', u'spleen', u'inflammation', u'concussion', u'nerve', u'cranial', u'dysfunction', u'rupture', u'asphyxia', u'multiple', u'cancerous', u'catastrophe', u'cerebral']
#
#Cluster 6
#[u'diving', u'divers']
#
#Cluster 7
#[u'sealed', u'boxes', u'ballots', u'box', u'sheets', u'computers', u'annexes']
#
#Cluster 8
#[u'zurab', u'gaytayev', u'akhmatkhanov', u'vakhit', u'vakhayev', u'temergeriyev', u'alikhadzhiyev', u'ruslanbek', u'ustarkhanov', u'ruslan', u'musa', u'balavdi', u'artur', u'magomadov', u'ilyasov']
#
#Cluster 9
#[u'citing', u'referring', u'relying']


#### CBOW with NO Stop words Model and Classifier II
#Cluster 0
#[u'handed', u'dispatched']
#
#Cluster 1
#[u'feels', u'tells', u'volunteered']
#
#Cluster 2
#[u'magnitogorsk', u'ipc', u'slahiye', u'pervomaysk', u'makeyevka', u'mezen', u'livestock', u'disablement', u'zrenjanin', u'gorlivka', u'bereznivsky']
#
#Cluster 3
#[u'bled', u'scrotum', u'tube', u'hyperaemia', u'laceration', u'bluish', u'graze', u'eyeball', u'erythema', u'trunk', u'purple', u'haemorrhaging', u'lobe', u'hematoma', u'grazes', u'radial', u'socket', u'scabbed', u'patches', u'dense', u'ecchymosis', u'deformed', u'pink', u'testicle', u'eyelid', u'bleed', u'heel', u'scab', u'swellings']
#
#Cluster 4
#[u'tolerable', u'groundwater', u'fullest', u'accomplished', u'catastrophic', u'foresee', u'shed', u'ignorance', u'connecting', u'absurd', u'collision', u'theoretically']
#
#Cluster 5
#[u'se', u'lbro', u'alena', u'leary', u'vilanova', u'pejchal', u'pastor', u'ksenija', u'wojtyczek', u'silvis', u'mahoney', u'dmitry', u'lubarda', u'faris', u'robert', u'ranzoni', u'pauliine', u'vehabovi', u'turkovi', u'gabriele', u'serghides', u'pere', u'spano', u'fridrik', u'antoanella', u'armen', u'kj', u'yonko', u'carlo', u'georgios', u'kucsko', u'stadlmayer', u'grozev', u'pol', u'branko', u'dedov', u'iulia', u'harutyunyan', u'rti', u'gri', u'mits', u'motoc', u'jon', u'ofra', u'valeriu', u'georges']
#
#Cluster 6
#[u'prerequisites']
#
#Cluster 7
#[u'perustuslaki', u'grundlag', u'finlands', u'suomen', u'empower', u'brandenburg', u'coll', u'xv', u'pardons', u'satversme']
#
#Cluster 8
#[u'ring', u'underpants', u'hammer', u'taped', u'bottom', u'torn', u'mask', u'barrel', u'belt', u'recorder', u'glasses', u'tape', u'sack', u'charger', u'rope', u'trousers', u'pillowcase', u'adhesive', u'underwear', u'wires', u'bottle', u'pile', u'pockets', u'coat', u'razor', u'refrigerator', u'hair', u'plaster', u'wrapped', u'plastic', u'padlock', u'towel', u'cigarette', u'blank', u'shirt', u'cloth', u'bonnet', u'straight', u'angle', u'mailbox', u'boot', u'bushes']
#
#Cluster 9
#[u'sadykov']


########## CBOW with NO Stop words Model and Extra words removed
#Cluster 0
#[u'helsinki', u'foundation', u'defender', u'khrp', u'memorial']#
#
#Cluster 1
#[u'wejherowo', u'korzeb', u'toru', u'sac', u'midday', u'ceylan', u'complemented', u'ttingen', u'widnica', u'ordinances', u'abdulhakim', u'demi', u'fyodorov', u'abstandsgebot', u'wroclaw', u'freiburg', u'dossier', u'kleindienst', u'exculpatory', u'palfreeman', u'backdated', u'rsav', u'outlines', u'micha', u'ssc', u'waiving', u'zbigniew', u'arrives', u'kucheruk', u'geri', u'kalisz']
#
#Cluster 2
#[u'vii', u'deals']
#
#Cluster 3
#[u'limbs', u'thumb', u'knee', u'torso', u'abrasion', u'temple', u'forearms', u'scratch', u'thigh', u'buttocks', u'jaw', u'vein', u'tooth', u'membrane', u'lip', u'hips', u'vertebra', u'lips', u'bruising', u'slash', u'ankle', u'shoulders', u'ear', u'amputated', u'side', u'shoulder', u'shin', u'elbow', u'bruised', u'redness', u'scalp', u'numbness', u'bones', u'haematoma', u'eyebrow', u'soft', u'nail', u'forehead', u'forearm', u'thorax', u'extremities', u'scab', u'hip', u'cheek', u'stitched', u'armpit', u'chin', u'wrist', u'swelling', u'elbows']
#
#Cluster 4
#[u'uttered', u'dealings', u'tone']
#
#Cluster 5
#[u'waterway', u'waters', u'hook', u'boundary', u'port', u'dragon', u'milford', u'harbour']
#
#Cluster 6
#[u'argue', u'conclusively']
#
#Cluster 7
#[u'counted', u'deducted', u'calculated', u'aggregate', u'reduced']
#
#Cluster 8
#[u'demirbank', u'debit', u'undeclared', u'sberbank', u'auctioning', u'cheque']
#
#Cluster 9
#[u'meetings', u'summonsed', u'attending', u'arranged', u'sessions', u'attended']

#CBOW_
#Cluster 0
#[u'norsk', u'rice', u'soci', u'warsame', u'ood', u'collegium', u'versa', u'cayman', u'fades', u'lee', u'zhakiyanov', u'generaal', u'qb', u'srebrenica', u'counties', u'zen', u'jrs', u'fl', u'collins', u'mmande', u'omst', u'njc', u'zealand', u'doherty', u'berisha', u'kenneth', u'westphalia', u'plotnikova', u'hardial', u'atak', u'recht', u'enforcements', u'diocese', u'maslov', u'opponent', u'vaiko', u'oskar', u'burhaan', u'morris', u'ndigheter', u'novruz', u'palfreeman', u'unterbringung', u'jurisprudentie', u'sakit', u'mazhiyeva', u'ns', u'baranja', u'chaired', u'vasiliauskas', u'galymzhan', u'zozulya', u'senator', u'wlr', u'rtv', u'executor', u'tat', u'savriddin']
#
#Cluster 1
#[u'relevance', u'resolve']
#
#Cluster 2
#[u'detectors', u'explosions', u'packets', u'angry', u'container', u'punch', u'glasses', u'launcher', u'tickets', u'charred', u'boxes', u'stain', u'trailer', u'filmed', u'sketch', u'missile', u'bakery', u'uniformed', u'parades', u'scuffle', u'screams', u'presumably', u'tripped', u'fort', u'unwell', u'ride', u'eneyeva', u'wallet', u'sounds', u'layijov', u'shells', u'metro', u'quarrel', u'burnt', u'rob', u'watching', u'mates', u'resembling', u'strangling', u'nails', u'grab', u'cloth', u'bankova', u'scared']
#
#Cluster 3
#[u'da', u'du']
#
#Cluster 4
#[u'incorrectly', u'supplement', u'annul', u'respondents']
#
#Cluster 5
#[u'locked', u'distance', u'noise', u'furniture', u'electric', u'bars']
#
#Cluster 6
#[u'freeze', u'rehabilitate', u'kyprianou', u'curiosity', u'standardised', u'wants', u'births', u'remarkable', u'analytical', u'irresponsibility', u'implements', u'manipulated', u'dealings', u'interviewees', u'strengthened', u'dish', u'imaginary', u'interceptions', u'convenient', u'casual', u'precarious', u'sfg', u'catastrophic', u'informer', u'businessmen', u'millions', u'collaborating', u'defense', u'delusions', u'cmsc', u'misunderstanding', u'courtrooms', u'meted', u'sporadically', u'happens', u'estimates', u'thesis', u'hiring', u'forceful', u'undeniably', u'exhibit', u'mentions', u'discussing', u'blog', u'comparatively', u'pertained', u'unique', u'exploiting', u'burials', u'grouped', u'foetus', u'swift']
#
#Cluster 7
#[u'abuses', u'among']
#
#Cluster 8
#[u'mihaela', u'lyubov', u'goran', u'tarasov', u'aleksandar', u'adamovi', u'gl', u'sterian', u'mircea', u'ari', u'mikhaylov', u'georgi', u'elik', u'magyar', u'atanasov']
#
#Cluster 9
#[u'ask']

#SG_
#Cluster 0
#[u'mount', u'attendance', u'understands', u'defended', u'choosing', u'assistance', u'translator', u'accusation', u'defence', u'defend', u'free', u'representation', u'require', u'legal', u'interpreter', u'obtain']
#
#Cluster 1
#[u'rebellion', u'individuals', u'zulkhumor', u'riots', u'rioting', u'revolt', u'concerted', u'suppress', u'target', u'actively', u'gangs', u'mob', u'aiming', u'organizing', u'indiscriminately', u'hazing', u'instigators', u'bullying', u'peacefully', u'khamdamova', u'deadly', u'dedy', u'sabotage', u'pretext', u'fugitives', u'procession', u'combat', u'disrupting', u'hostage', u'tactics']
#
#Cluster 2
#[u'campobasso', u'reaffirmed', u'ble', u'pupino', u'nieces', u'disenfranchisement', u'davis', u'qazi', u'runa', u'fsia', u'auld', u'ukpc', u'cl', u'disputing', u'bryan', u'retstidende', u'verfassungsgerichtshof', u'braga', u'blake', u'officium', u'bezwaar', u'hesse', u'summarized', u'fmo', u'khudobin', u'ergezen', u'uksc', u'queen', u'accede', u'dirdizov', u'revisited', u'almir', u'dissented', u'khudoyorov', u'jurisprudentie', u'tot', u'mauritius', u'chassagnou', u'ecj', u'begum', u'fac', u'pinochet', u'rtv']
#
#Cluster 3
#[u'zempl', u'rl', u'starosta', u'magnitogorsk', u'kurulu', u'mitrovica', u'evo', u'jel', u'zhovti', u'aniela', u'manchester', u'zef', u'dm', u'slovenj', u'lcea', u'genel', u'mnicu', u'tulcea', u'nad', u'beograd', u'sandomierz', u'zurich', u'jesenice', u'aba', u'gabrovo', u'seaport', u'uglegorskiy', u'maramure', u'melnikova', u'af', u'kocaeli', u'klagenfurt', u'valsts', u'zrenjanin', u'radomsko', u'nis', u'gradec', u'vody', u'borsod', u'grishchenko', u'delta', u'awy', u'krupanj']
#
#Cluster 4
#[u'akhmatov', u'makarov', u'fatima', u'khadizhat', u'kirova', u'shaipova', u'spelled', u'magomadova', u'roza', u'askhabova', u'ismailova', u'korolev', u'aweys', u'ibragimova', u'musayeva', u'aliyeva', u'jeli', u'zara', u'madina', u'abdurashidova', u'khava', u'kk', u'raisa', u'cabdulaahi', u'kheda', u'mukhtarova', u'mazhiyeva', u'zulay', u'kapanadze', u'lagutin', u'chiri', u'dzhabrailova', u'malika']
#
#Cluster 5
#[u'pulse', u'hole', u'ring', u'stripe', u'bit', u'bottom', u'tube', u'uneven', u'pole', u'stone', u'liquid', u'shoe', u'belt', u'casing', u'holes', u'bend', u'glasses', u'railing', u'spots', u'charred', u'millimetres', u'syringe', u'bitten', u'stained', u'stain', u'heels', u'electrical', u'shackled', u'toenails', u'spotted', u'slowly', u'crushed', u'exited', u'shaved', u'radiator', u'burn', u'wires', u'residue', u'stains', u'razor', u'cut', u'socks', u'breathe', u'mm', u'plaster', u'exterior', u'bandaged', u'shards', u'bandage', u'fainted', u'cigarette', u'asphalt', u'edge', u'canal', u'cloth', u'barely', u'bending', u'brush']
#
#Cluster 6
#[u'feasibility', u'classify', u'remind', u'reassess', u'obliging', u'confine', u'defaulting', u'disregarding', u'efficiently', u'detailing', u'failings', u'basically', u'substituted', u'halt', u'impossible', u'cep', u'completeness', u'commences', u'rosdorf', u'assigning', u'pointless', u'inefficient', u'sake', u'impracticable', u'onward', u'esa', u'preparations', u'impossibility', u'preventative', u'obsolete', u'closure', u'unqualified']
#
#Cluster 7
#[u'significance', u'dimension', u'sense', u'exaggerated', u'fundamentally', u'wider', u'disadvantage', u'balanced', u'topic', u'criterion', u'implications', u'comes', u'lacks', u'utmost', u'distinction', u'overriding', u'admittedly', u'doctrine', u'important', u'ambiguous', u'greater', u'emphasises', u'approach', u'core', u'differences', u'goes', u'spirit', u'seems', u'reflect', u'conceivable', u'weigh', u'respects', u'broadly']
#
#Cluster 8
#[u'bodyguards', u'kremlin', u'boarded', u'mashtots', u'bremen', u'bombed', u'smuggle', u'unexploded', u'guarding', u'lounge', u'usukhchay', u'bb', u'pedestrians', u'ismayilli', u'cordons', u'biryukov', u'perimeter', u'karakert', u'locals', u'signal', u'shopping', u'museum', u'marketplace', u'waterfront', u'yakimanka', u'reconnaissance', u'bolshoy', u'cscoc', u'buses', u'missiles', u'metro', u'triumfalnaya', u'raids', u'disarmed', u'vip', u'patrols', u'coordinates', u'ssu', u'frigate', u'tata', u'improvised', u'badges', u'siderov']
#
#Cluster 9
#[u'rp', u'rice', u'reunion', u'ghana', u'osar', u'mediterranean', u'reunited', u'campione', u'qatar', u'tunisians', u'cara', u'centuries', u'venezuela', u'zealand', u'cspa', u'mcdonald', u'diya', u'copenhagen', u'sfh', u'undocumented', u'sponsors', u'musician', u'expatriate', u'abkhazia', u'chicago', u'buddhist']

#CBOW_NSX
#Cluster 0
#[u'appropriately', u'structural', u'task', u'morality', u'fairly', u'expectation', u'recent', u'permits', u'forcible', u'pursuing', u'accepts', u'certainly', u'highlighted', u'lacking', u'accessibility', u'fears', u'exhaustive', u'characterised', u'prevents', u'undermined', u'examples', u'emphasis', u'ways', u'careful', u'seems', u'recognise', u'albeit']
#
#Cluster 1
#[u'tabanovce', u'asgarov', u'sergeyeva', u'counterintelligence', u'sior', u'counterparts', u'resided', u'liaison', u'addresses', u'idp', u'mujahedin', u'lent', u'subdivision', u'muledal', u'ubk', u'khodorkovskiy', u'fought', u'rie']
#
#Cluster 2
#[u'cooking', u'leaning', u'hammer', u'bit', u'smashed', u'explosions', u'wood', u'container', u'blinds', u'vision', u'screen', u'breath', u'folding', u'slats', u'cavity', u'stone', u'burning', u'shops', u'plank', u'bowl', u'subarachnoid', u'pyelonephritis', u'focal', u'cabins', u'forearms', u'bluish', u'experiencing', u'punch', u'baby', u'bent', u'occipital', u'glasses', u'footwear', u'reactivation', u'illuminated', u'flush', u'launcher', u'unheated', u'basin', u'nurses', u'disc', u'infirmary', u'bread', u'thick', u'respiratory', u'regained', u'pipe', u'hematoma', u'breakfast', u'renal', u'stain', u'punches', u'soil', u'vent', u'heated', u'mirror', u'loose', u'dilapidated', u'rats', u'injection', u'tomography', u'humid', u'nausea', u'panes', u'cutlery', u'bakery', u'ankles', u'upset', u'haemorrhages', u'curtains', u'emptied', u'ischaemic', u'pans', u'floors', u'anus', u'encephalopathy', u'squat', u'tripped', u'cranial', u'haseki', u'lice', u'cupboard', u'wires', u'haematuria', u'tumour', u'deep', u'coughing', u'intracranial', u'hurting', u'edges', u'vascular', u'rupture', u'frames', u'batons', u'muradova', u'basins', u'unventilated', u'curtain', u'stuffy', u'pin', u'asthenia', u'games', u'towels', u'cockroaches', u'catch', u'lamps', u'bats', u'cafeteria', u'excoriation', u'partitions', u'stiflingly', u'bulbs', u'watching', u'twisting', u'mates', u'chained', u'bedside', u'quiet', u'helmet', u'antibodies', u'towel', u'strangling', u'nails', u'fish', u'belts', u'row', u'breathalyser', u'cupboards', u'blank', u'kucheruk', u'edge', u'nasal', u'colorectal', u'cloth', u'bell', u'puncture', u'locking', u'maternal', u'straight', u'osteochondrosis', u'barely', u'duodenal']
#
#Cluster 3
#[u'omissions', u'delay', u'limits', u'effect', u'comply', u'basis', u'specified', u'obliged', u'sanction', u'imposition', u'must', u'limitation', u'impose', u'fails', u'competent', u'speedily', u'unjustified']
#
#Cluster 4
#[u'writings', u'precincts', u'hyperactivity', u'rumour', u'circulated', u'briefed', u'groundwater', u'myopia', u'flood', u'bribed', u'tankers', u'abortion', u'rehabilitate', u'informers', u'seldom', u'kyprianou', u'curiosity', u'reprisal', u'spill', u'countered', u'impressions', u'intoxicating', u'concentrated', u'auditory', u'wants', u'qamishli', u'faster', u'particles', u'poisonous', u'ethiopia', u'rioting', u'reward', u'exterminate', u'informants', u'mink', u'survey', u'mounting', u'illustrated', u'obesity', u'odd', u'hereditary', u'scepticism', u'web', u'palliative', u'necessities', u'exchanges', u'nuclear', u'perinatal', u'implicate', u'overseeing', u'imaginary', u'placards', u'alb', u'uncomfortable', u'machines', u'toll', u'thrust', u'antics', u'precarious', u'latent', u'parks', u'toys', u'renovating', u'dance', u'flammable', u'intellectually', u'collaborate', u'reacting', u'fires', u'concentrating', u'tragic', u'rezina', u'quotas', u'ordinate', u'generalised', u'restless', u'randomly', u'unintentionally', u'waters', u'dubai', u'culturally', u'rested', u'kitty', u'volatile', u'collaborating', u'ekbacken', u'documenting', u'agendas', u'beards', u'temporal', u'demeanour', u'perfectly', u'minds', u'amisom', u'corke', u'appearances', u'lately', u'phenomena', u'helplessness', u'leasing', u'paedophilia', u'reader', u'witnessing', u'jam', u'annoyed', u'sorts', u'dushanbe', u'wanting', u'notorious', u'intact', u'atrocities', u'digital', u'targets', u'blurred', u'confessing', u'combatting', u'graves', u'associates', u'traffickers', u'paz', u'delirium', u'audience', u'routines', u'notable', u'sedative', u'leaked', u'fill', u'retiring', u'stir', u'psychomotor', u'analgesic', u'leaking', u'irregularly', u'newborn', u'raw', u'circumcision', u'displace', u'strongest', u'opiates', u'educators', u'specifications', u'wedding', u'looks', u'exploited', u'videos', u'peacefully', u'isfara', u'wise', u'appetite', u'pride', u'abductor', u'dressing', u'aerobatic', u'thesis', u'hiring', u'forceful', u'shizo', u'opioid', u'reconstruct', u'poarta', u'exhibit', u'degradation', u'mentions', u'rising', u'kazimov', u'winner', u'platform', u'fetched', u'workshops', u'dedy', u'sabotage', u'discussing', u'subscriber', u'unhealthy', u'embezzled', u'viewers', u'symbol', u'carers', u'seemingly', u'unique', u'coincided', u'deployment', u'patriotic', u'burials', u'enhancing', u'unsealed', u'coordinator', u'assassinations', u'businesses', u'imams', u'explore', u'swear', u'impulsive', u'contaminated', u'diagnostics', u'commands', u'komletinci', u'breeding', u'recurrent', u'approximate']
#
#Cluster 5
#[u'poalelungi', u'ini', u'bianku', u'de', u'vu', u'ledi', u'mihai', u'neboj', u'gaetano']
#
#Cluster 6
#[u'neurologist', u'drawn', u'physician', u'test', u'depth', u'hospitalised', u'discharged', u'neurological', u'detected', u'clinic', u'closed']
#
#Cluster 7
#[u'qaeda', u'corps', u'newspaper', u'dh', u'croat', u'delegates', u'editor', u'hezb', u'meeting', u'pen', u'serb', u'christian', u'bosniac']
#
#Cluster 8
#[u'extend', u'barred', u'reasoned', u'ordering', u'terminate', u'terminated', u'extending', u'discontinue', u'expired', u'receipt', u'defendant', u'expiry', u'extension', u'limit']
#
#Cluster 9
#[u'silver', u'plate', u'pocket', u'cordoned', u'balaclava', u'torn', u'handcuff', u'zil', u'bedroom', u'butts', u'wore', u'green', u'minibus', u'ifvs', u'lada', u'throw', u'streets', u'autumn', u'unconscious', u'mihaylov', u'mosque', u'booth', u'arrive', u'pulling', u'photograph', u'kick', u'pistols', u'tailgate', u'banged', u'loaded', u'bottles', u'adhesive', u'camouflaged', u'bmw', u'oil', u'gates', u'zakan', u'rifles', u'burned', u'lorries', u'keys', u'priora', u'swearing', u'kalashnikov', u'portable', u'pick', u'mother', u'dinner', u'balaclavas', u'sacks', u'tyre', u'helicopter', u'gunshots', u'passed', u'cigarette', u'fragments', u'tinted', u'minivans', u'tabletka', u'lobby', u'carrier', u'blown']

In [8]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


In [9]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["sentences"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["sentences"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1
    
    
## Fit a random forest and extract predictions 
#forest = RandomForestClassifier(n_estimators = 100)
#
## Fitting the forest may take a few minutes
#print "Fitting a random forest to labeled training data..."
#forest = forest.fit(train_centroids,train["AV3"])
#result = forest.predict(test_centroids)

#from sklearn.grid_search import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier

grid_values = {} #{'C':[30]} # Decide which settings you want for the grid search. 

model_LR = GridSearchCV(RandomForestClassifier(n_estimators = 100), 
                        grid_values, 
                        scoring = 'roc_auc', cv = 20) 

# Try to set the scoring on what the contest is asking for. 
# The contest says scoring is for area under the ROC curve, so use this.
                            
model_LR.fit(train_centroids, train["AV3"] ) # Fit the model.

model_LR.grid_scores_



## Write the test results 
#output = pd.DataFrame(data={"title":test["title"],"conclusion":test["conclusion"], "AV3":result})
##output.to_csv( "BagOfCentroids500.csv", index=False, quoting=3, escapechar='\\' , sep='^')
#output.to_csv( "CBOW_BagOfCentroids500.csv", index=False, quoting=3, escapechar='\\' , sep='^')
LR_result = model_LR.predict_proba(test_centroids)[:,1] 
#LR_output = pd.DataFrame(data={"id":test["id"], "AV3":LR_result}) # Create our dataframe that will be written.
LR_output = pd.DataFrame( data={"title":test["title"],"conclusion":test["conclusion"], "AV3_Predict":LR_result} )
#LR_output.to_csv('Logistic_Reg_Proj2.csv', index=False, quoting=3) # Get the .csv file we will submit to Kaggle.
LR_output.to_csv( "A3008_SG_CENT_NSX.csv", index=False, quoting=3, escapechar='\\', sep='^'  )
